In [ ]:
import pandas as pd
import math
import numpy as np

In [ ]:
df = pd.read_excel("/content/IEOR4004_HW3_MobileCareData.xlsx", sheet_name=None)

In [ ]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 62.5 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp

In [ ]:
from gurobipy import Model, GRB, quicksum, Env

In [ ]:
from google.colab import userdata

# Create an environment with your WLS license
params = {
"WLSACCESSID": "206b3829-27fe-4e85-b2b5-40f61b2d7366",
"WLSSECRET": "f86b22c6-80f1-49ab-a97a-a3ad7bf5149a",
"LICENSEID": 2709572
}

env = Env(params=params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2709572
Academic license 2709572 - for non-commercial use only - registered to mg___@columbia.edu


# Introduction

Mobile C.A.R.E. Foundation (MCF) provides asthma management services to elementary schoolchildren from low-income families in the greater Chicago area using mobile asthma vans that operate on weekdays. Due to a recent funding shortfall, MCF has reduced its fleet and must now make sharper resource-allocation decisions to maintain service effectiveness across the 19 schools it serves—especially for higher-severity patients who may face greater health risk if care is delayed.

In this project, I use MCF’s provided datasets (school locations, patient lists, patient severity levels, and historical appointment logs by school) to (1) conduct exploratory data analysis to estimate key operational parameters such as demand by severity and inferred van capacity, (2) document modeling assumptions and identify additional data that would improve realism, and (3) formulate and solve a monthly two-van scheduling and assignment optimization model. The goal is to determine which schools each van should serve, how many appointment slots each van should provide, and which patients should be scheduled next month, leveraging historical fields such as no-shows, due-back timing, and utilization indicators (hospitalizations, ER visits, and days missed) to support a clinically meaningful and operationally feasible plan.


## 1. EDA

In [ ]:
print(df.keys())                                   # sheet names

dict_keys(['Explanations', 'School Locations', 'Case_PatientList', 'Case_PatientSeverity', 'Case_SCH#0_APPS', 'Case_SCH#1_APPS', 'Case_SCH#2_APPS', 'Case_SCH#3_APPS', 'Case_SCH#4_APPS', 'Case_SCH#5_APPS', 'Case_SCH#6_APPS', 'Case_SCH#7_APPS', 'Case_SCH#8_APPS', 'Case_SCH#9_APPS', 'Case_SCH#10_APPS', 'Case_SCH#11_APPS', 'Case_SCH#12_APPS', 'Case_SCH#13_APPS', 'Case_SCH#14_APPS', 'Case_SCH#15_APPS', 'Case_SCH#16_APPS', 'Case_SCH#17_APPS', 'Case_SCH#18_APPS', 'Case_SCH#19_APPS'])


In [ ]:
df_School_Locations = df[list(df.keys())[1]]

In [ ]:
df_Case_PatientList = df[list(df.keys())[2]]

In [ ]:
# Convert to long format — keep original values
df_long = df_Case_PatientList.melt(
    var_name="school_name",
    value_name="patient_full"
).dropna()

# Simply rename columns for clarity — no ID extraction or modification
df_patients = df_long.rename(columns={
    "patient_full": "patient_id",
    "school_name": "school_id"
}).drop_duplicates()

In [ ]:
df_Case_PatientSeverity = df[list(df.keys())[3]]

In [ ]:
df_Case_PatientSeverity.head()

,PATIENT,SEVERITY
0,Patient #0,MildPersistent
1,Patient #1,MildPersistent
2,Patient #2,ModeratePersistent
3,Patient #3,MildPersistent
4,Patient #4,MildPersistent


check what levels are in there

In [ ]:
df_Case_PatientSeverity['SEVERITY'].unique()

array(['MildPersistent', 'ModeratePersistent', 'SeverePersistent',
       'MildIntermittent'], dtype=object)

In [ ]:
df_Case_SCH_0_APPS =  df['Case_SCH#0_APPS']

In [ ]:
df_Case_SCH_0_APPS.dtypes

,0
VAN,object
APP DATE,datetime64[ns]
APP TIME,object
PATIENT,object
LAST SEEN,datetime64[ns]
LAST DIAG,object
LAST DUE BACK,datetime64[ns]
CURRENT DIAG,object
DUE BACK,datetime64[ns]
COMMENT,object


In [ ]:
df_Case_SCH_0_APPS.head()

,VAN,APP DATE,APP TIME,PATIENT,LAST SEEN,LAST DIAG,LAST DUE BACK,CURRENT DIAG,DUE BACK,COMMENT,HOSP.,ER-VISIT,DAYS MISSED
0,Van #0,2012-01-11,09:00:00,Patient #1575,2011-09-28,Improved,2011-10-28,NO SHOW,2011-10-28,NaN,-,-,-
1,Van #0,2012-01-11,09:30:00,Patient #1465,2011-09-28,Controlled,2011-12-28,Controlled,2012-04-11,NaN,0,0,0
2,Van #0,2012-01-11,10:00:00,Patient #1001,2011-12-07,Improved,2012-01-07,Controlled,2012-04-11,NaN,0,0,2
3,Van #0,2012-01-11,10:30:00,Patient #1520,2011-12-07,Improved,2012-01-07,Controlled,2012-04-11,NaN,0,0,0
4,Van #0,2012-01-11,11:00:00,Patient #497,2011-11-02,Controlled,2012-02-02,Controlled,2012-04-11,NaN,0,0,0


In [ ]:
df_Case_SCH_1_APPS = df['Case_SCH#1_APPS']
df_Case_SCH_2_APPS = df['Case_SCH#2_APPS']
df_Case_SCH_3_APPS = df['Case_SCH#3_APPS']
df_Case_SCH_4_APPS = df['Case_SCH#4_APPS']
df_Case_SCH_5_APPS = df['Case_SCH#5_APPS']
df_Case_SCH_6_APPS = df['Case_SCH#6_APPS']
df_Case_SCH_7_APPS = df['Case_SCH#7_APPS']
df_Case_SCH_8_APPS = df['Case_SCH#8_APPS']
df_Case_SCH_9_APPS = df['Case_SCH#9_APPS']
df_Case_SCH_10_APPS = df['Case_SCH#10_APPS']
df_Case_SCH_11_APPS = df['Case_SCH#11_APPS']
df_Case_SCH_12_APPS = df['Case_SCH#12_APPS']
df_Case_SCH_13_APPS = df['Case_SCH#13_APPS']
df_Case_SCH_14_APPS = df['Case_SCH#14_APPS']
df_Case_SCH_15_APPS = df['Case_SCH#15_APPS']
df_Case_SCH_16_APPS = df['Case_SCH#16_APPS']
df_Case_SCH_17_APPS = df['Case_SCH#17_APPS']
df_Case_SCH_18_APPS = df['Case_SCH#18_APPS']
df_Case_SCH_19_APPS = df['Case_SCH#19_APPS']

### Estimate the capacity of each Van


Historically, Van 0 has serviced schools with #even number as their labels, and Van 1 has servied schools with #odd number as their labels. I assume this approach is purely random.

Van 0 appointments by day per school:

In [ ]:
# Count slots per (APP DATE) for even schedule numbers

slots_per_day_0_0 = df_Case_SCH_0_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 0:", slots_per_day_0_0['slots'].unique())

slots_per_day_0_2 = df_Case_SCH_2_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 2:", slots_per_day_0_2['slots'].unique())

slots_per_day_0_4 = df_Case_SCH_4_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 4:", slots_per_day_0_4['slots'].unique())

slots_per_day_0_6 = df_Case_SCH_6_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 6:", slots_per_day_0_6['slots'].unique())

slots_per_day_0_8 = df_Case_SCH_8_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 8:", slots_per_day_0_8['slots'].unique())

slots_per_day_0_10 = df_Case_SCH_10_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 10:", slots_per_day_0_10['slots'].unique())

slots_per_day_0_12 = df_Case_SCH_12_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 12:", slots_per_day_0_12['slots'].unique())

slots_per_day_0_14 = df_Case_SCH_14_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 14:", slots_per_day_0_14['slots'].unique())

slots_per_day_0_16 = df_Case_SCH_16_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 16:", slots_per_day_0_16['slots'].unique())

slots_per_day_0_18 = df_Case_SCH_18_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 18:", slots_per_day_0_18['slots'].unique())


SCH 0: [16]
SCH 2: [16]
SCH 4: [16]
SCH 6: [16]
SCH 8: [16]
SCH 10: [16]
SCH 12: [16]
SCH 14: [16]
SCH 16: [16]
SCH 18: [16]


Were any schools served on the same date for Van 0?

In [ ]:
# Which school indices to check
even_ids = list(range(0, 20, 2))

from functools import reduce

# Collect the DataFrames from your variables if they exist
dfs_even = {}
missing = []
for i in even_ids:
    varname = f"df_Case_SCH_{i}_APPS"
    if varname in globals():
        df_i = globals()[varname].copy()
        # ensure datetime, then normalize to date-only (drop time component)
        df_i['APP DATE'] = pd.to_datetime(df_i['APP DATE'], errors='coerce')
        df_i['APP_DATE_ONLY'] = df_i['APP DATE'].dt.normalize()
        dfs_even[i] = df_i
    else:
        missing.append(varname)

if missing:
    print("Warning: missing DataFrames:", missing)

# Build a set of dates for each even-numbered school
dates_by_school = {
    i: set(dfs_even[i].loc[dfs_even[i]['APP_DATE_ONLY'].notna(), 'APP_DATE_ONLY'].unique())
    for i in dfs_even
}

# If we have at least one school, compute intersection across all even-numbered schools
if dates_by_school:
    common_dates = reduce(lambda a, b: a & b, dates_by_school.values())
else:
    common_dates = set()

# Results
all_even_share_a_date = len(common_dates) > 0
print("Do ALL even-numbered schools share at least one APP DATE?", all_even_share_a_date)

if all_even_share_a_date:
    print("Common APP DATE(s) across ALL even-numbered schools:")
    for d in sorted(common_dates):
        print(d.date())  # prints YYYY-MM-DD


Do ALL even-numbered schools share at least one APP DATE? False


Van 0 appointment days by month per school



In [ ]:
# ---------- SCHOOL 0 ----------
df_Case_SCH_0_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_0_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_0 = (
    df_Case_SCH_0_APPS.groupby(df_Case_SCH_0_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_0['month'] = days_per_month_0['APP DATE'].dt.strftime('%Y-%m')
days_per_month_0 = days_per_month_0[['month', 'clinic_days']]
print("School 0:")
print(days_per_month_0)

# ---------- SCHOOL 2 ----------
df_Case_SCH_2_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_2_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_2 = (
    df_Case_SCH_2_APPS.groupby(df_Case_SCH_2_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_2['month'] = days_per_month_2['APP DATE'].dt.strftime('%Y-%m')
days_per_month_2 = days_per_month_2[['month', 'clinic_days']]
print("\nSchool 2:")
print(days_per_month_2)

# ---------- SCHOOL 4 ----------
df_Case_SCH_4_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_4_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_4 = (
    df_Case_SCH_4_APPS.groupby(df_Case_SCH_4_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_4['month'] = days_per_month_4['APP DATE'].dt.strftime('%Y-%m')
days_per_month_4 = days_per_month_4[['month', 'clinic_days']]
print("\nSchool 4:")
print(days_per_month_4)

# ---------- SCHOOL 6 ----------
df_Case_SCH_6_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_6_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_6 = (
    df_Case_SCH_6_APPS.groupby(df_Case_SCH_6_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_6['month'] = days_per_month_6['APP DATE'].dt.strftime('%Y-%m')
days_per_month_6 = days_per_month_6[['month', 'clinic_days']]
print("\nSchool 6:")
print(days_per_month_6)

# ---------- SCHOOL 8 ----------
df_Case_SCH_8_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_8_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_8 = (
    df_Case_SCH_8_APPS.groupby(df_Case_SCH_8_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_8['month'] = days_per_month_8['APP DATE'].dt.strftime('%Y-%m')
days_per_month_8 = days_per_month_8[['month', 'clinic_days']]
print("\nSchool 8:")
print(days_per_month_8)

# ---------- SCHOOL 10 ----------
df_Case_SCH_10_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_10_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_10 = (
    df_Case_SCH_10_APPS.groupby(df_Case_SCH_10_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_10['month'] = days_per_month_10['APP DATE'].dt.strftime('%Y-%m')
days_per_month_10 = days_per_month_10[['month', 'clinic_days']]
print("\nSchool 10:")
print(days_per_month_10)

# ---------- SCHOOL 12 ----------
df_Case_SCH_12_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_12_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_12 = (
    df_Case_SCH_12_APPS.groupby(df_Case_SCH_12_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_12['month'] = days_per_month_12['APP DATE'].dt.strftime('%Y-%m')
days_per_month_12 = days_per_month_12[['month', 'clinic_days']]
print("\nSchool 12:")
print(days_per_month_12)

# ---------- SCHOOL 14 ----------
df_Case_SCH_14_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_14_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_14 = (
    df_Case_SCH_14_APPS.groupby(df_Case_SCH_14_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_14['month'] = days_per_month_14['APP DATE'].dt.strftime('%Y-%m')
days_per_month_14 = days_per_month_14[['month', 'clinic_days']]
print("\nSchool 14:")
print(days_per_month_14)

# ---------- SCHOOL 16 ----------
df_Case_SCH_16_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_16_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_16 = (
    df_Case_SCH_16_APPS.groupby(df_Case_SCH_16_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_16['month'] = days_per_month_16['APP DATE'].dt.strftime('%Y-%m')
days_per_month_16 = days_per_month_16[['month', 'clinic_days']]
print("\nSchool 16:")
print(days_per_month_16)

# ---------- SCHOOL 18 ----------
df_Case_SCH_18_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_18_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_18 = (
    df_Case_SCH_18_APPS.groupby(df_Case_SCH_18_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_18['month'] = days_per_month_18['APP DATE'].dt.strftime('%Y-%m')
days_per_month_18 = days_per_month_18[['month', 'clinic_days']]
print("\nSchool 18:")
print(days_per_month_18)


School 0:
      month  clinic_days
0   2012-01            2
1   2012-02            1
2   2012-03            2
3   2012-04            2
4   2012-05            2
5   2012-06            1
6   2012-07            2
7   2012-08            2
8   2012-09            1
9   2012-10            2
10  2012-11            2
11  2012-12            2
12  2013-01            2
13  2013-02            1
14  2013-03            2
15  2013-04            2
16  2013-05            1
17  2013-06            2
18  2013-07            2
19  2013-08            2
20  2013-09            1
21  2013-10            2
22  2013-11            2
23  2013-12            2

School 2:
      month  clinic_days
0   2012-01            2
1   2012-02            1
2   2012-03            2
3   2012-04            2
4   2012-05            1
5   2012-06            2
6   2012-07            2
7   2012-08            2
8   2012-09            1
9   2012-10            2
10  2012-11            2
11  2012-12            2
12  2013-01            1
13  

The number of clinic days across all schools per month for Van 0:

In [ ]:
even_df_names = [
    'days_per_month_0','days_per_month_2','days_per_month_4',
    'days_per_month_6','days_per_month_8','days_per_month_10',
    'days_per_month_12','days_per_month_14','days_per_month_16',
    'days_per_month_18'
]

even_dfs = []
for name in even_df_names:
    if name in globals():
        even_dfs.append(globals()[name].copy())
    else:
        print(f"Warning: {name} not found — skipping")

if not even_dfs:
    raise ValueError("No even school DataFrames found.")

# Stack them, then sum per month
combined = pd.concat(even_dfs, ignore_index=True)
clinic_days_total = (
    combined.groupby('month', as_index=False)['clinic_days']
            .sum()
            .rename(columns={'clinic_days': 'total_clinic_days'})
            .sort_values('month')
)

print(clinic_days_total)



      month  total_clinic_days
0   2012-01                 18
1   2012-02                 17
2   2012-03                 17
3   2012-04                 17
4   2012-05                 18
5   2012-06                 17
6   2012-07                 18
7   2012-08                 18
8   2012-09                 16
9   2012-10                 19
10  2012-11                 17
11  2012-12                 17
12  2013-01                 18
13  2013-02                 16
14  2013-03                 17
15  2013-04                 18
16  2013-05                 18
17  2013-06                 16
18  2013-07                 19
19  2013-08                 17
20  2013-09                 17
21  2013-10                 18
22  2013-11                 17
23  2013-12                 18


**From the above results, we know that, historically, Van 0 serves 16 appoints per day for each school, only one school on the same date, maximum 2 clinic days and minimum 1 clinic day per month per school, and 19 clinical days maximum 16 clinical days minimum for all schools per month.**

Van 1:

In [ ]:
# Count slots per (APP DATE) for odd schedule numbers
slots_per_day_0_1 = df_Case_SCH_1_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 1:", slots_per_day_0_1['slots'].unique())

slots_per_day_0_3 = df_Case_SCH_3_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 3:", slots_per_day_0_3['slots'].unique())

slots_per_day_0_5 = df_Case_SCH_5_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 5:", slots_per_day_0_5['slots'].unique())

slots_per_day_0_7 = df_Case_SCH_7_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 7:", slots_per_day_0_7['slots'].unique())

slots_per_day_0_9 = df_Case_SCH_9_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 9:", slots_per_day_0_9['slots'].unique())

slots_per_day_0_11 = df_Case_SCH_11_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 11:", slots_per_day_0_11['slots'].unique())

slots_per_day_0_13 = df_Case_SCH_13_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 13:", slots_per_day_0_13['slots'].unique())

slots_per_day_0_15 = df_Case_SCH_15_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 15:", slots_per_day_0_15['slots'].unique())

slots_per_day_0_17 = df_Case_SCH_17_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 17:", slots_per_day_0_17['slots'].unique())

slots_per_day_0_19 = df_Case_SCH_19_APPS.groupby(["APP DATE"]).size().reset_index(name="slots")
print("SCH 19:", slots_per_day_0_19['slots'].unique())


SCH 1: [16]
SCH 3: [16]
SCH 5: [16]
SCH 7: [16]
SCH 9: [16]
SCH 11: [16]
SCH 13: [16]
SCH 15: [16]
SCH 17: [16]
SCH 19: [16]


Were any schools served on the same date for Van 1?

In [ ]:
# Which school indices to check
odd_ids = list(range(1, 21, 2))

from functools import reduce

# Collect the DataFrames from your variables if they exist
dfs_odd = {}
missing = []
for i in odd_ids:
    varname = f"df_Case_SCH_{i}_APPS"
    if varname in globals():
        df_i = globals()[varname].copy()
        # ensure datetime, then normalize to date-only (drop time component)
        df_i['APP DATE'] = pd.to_datetime(df_i['APP DATE'], errors='coerce')
        df_i['APP_DATE_ONLY'] = df_i['APP DATE'].dt.normalize()
        dfs_odd[i] = df_i
    else:
        missing.append(varname)

if missing:
    print("Warning: missing DataFrames:", missing)

# Build a set of dates for each odd-numbered school
dates_by_school = {
    i: set(dfs_odd[i].loc[dfs_odd[i]['APP_DATE_ONLY'].notna(), 'APP_DATE_ONLY'].unique())
    for i in dfs_odd
}

# If we have at least one school, compute intersection across all odd-numbered schools
if dates_by_school:
    common_dates = reduce(lambda a, b: a & b, dates_by_school.values())
else:
    common_dates = set()

# Results
all_odd_share_a_date = len(common_dates) > 0
print("Do ALL odd-numbered schools share at least one APP DATE?", all_odd_share_a_date)

if all_odd_share_a_date:
    print("Common APP DATE(s) across ALL odd-numbered schools:")
    for d in sorted(common_dates):
        print(d.date())  # prints YYYY-MM-DD

Do ALL odd-numbered schools share at least one APP DATE? False


In [ ]:
# ---------- SCHOOL 1 ----------
df_Case_SCH_1_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_1_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_1 = (
    df_Case_SCH_1_APPS.groupby(df_Case_SCH_1_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_1['month'] = days_per_month_1['APP DATE'].dt.strftime('%Y-%m')
days_per_month_1 = days_per_month_1[['month', 'clinic_days']]
print("\nSchool 1:")
print(days_per_month_1)

# ---------- SCHOOL 3 ----------
df_Case_SCH_3_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_3_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_3 = (
    df_Case_SCH_3_APPS.groupby(df_Case_SCH_3_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_3['month'] = days_per_month_3['APP DATE'].dt.strftime('%Y-%m')
days_per_month_3 = days_per_month_3[['month', 'clinic_days']]
print("\nSchool 3:")
print(days_per_month_3)

# ---------- SCHOOL 5 ----------
df_Case_SCH_5_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_5_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_5 = (
    df_Case_SCH_5_APPS.groupby(df_Case_SCH_5_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_5['month'] = days_per_month_5['APP DATE'].dt.strftime('%Y-%m')
days_per_month_5 = days_per_month_5[['month', 'clinic_days']]
print("\nSchool 5:")
print(days_per_month_5)

# ---------- SCHOOL 7 ----------
df_Case_SCH_7_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_7_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_7 = (
    df_Case_SCH_7_APPS.groupby(df_Case_SCH_7_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_7['month'] = days_per_month_7['APP DATE'].dt.strftime('%Y-%m')
days_per_month_7 = days_per_month_7[['month', 'clinic_days']]
print("\nSchool 7:")
print(days_per_month_7)

# ---------- SCHOOL 9 ----------
df_Case_SCH_9_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_9_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_9 = (
    df_Case_SCH_9_APPS.groupby(df_Case_SCH_9_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_9['month'] = days_per_month_9['APP DATE'].dt.strftime('%Y-%m')
days_per_month_9 = days_per_month_9[['month', 'clinic_days']]
print("\nSchool 9:")
print(days_per_month_9)

# ---------- SCHOOL 11 ----------
df_Case_SCH_11_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_11_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_11 = (
    df_Case_SCH_11_APPS.groupby(df_Case_SCH_11_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_11['month'] = days_per_month_11['APP DATE'].dt.strftime('%Y-%m')
days_per_month_11 = days_per_month_11[['month', 'clinic_days']]
print("\nSchool 11:")
print(days_per_month_11)

# ---------- SCHOOL 13 ----------
df_Case_SCH_13_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_13_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_13 = (
    df_Case_SCH_13_APPS.groupby(df_Case_SCH_13_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_13['month'] = days_per_month_13['APP DATE'].dt.strftime('%Y-%m')
days_per_month_13 = days_per_month_13[['month', 'clinic_days']]
print("\nSchool 13:")
print(days_per_month_13)

# ---------- SCHOOL 15 ----------
df_Case_SCH_15_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_15_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_15 = (
    df_Case_SCH_15_APPS.groupby(df_Case_SCH_15_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_15['month'] = days_per_month_15['APP DATE'].dt.strftime('%Y-%m')
days_per_month_15 = days_per_month_15[['month', 'clinic_days']]
print("\nSchool 15:")
print(days_per_month_15)

# ---------- SCHOOL 17 ----------
df_Case_SCH_17_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_17_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_17 = (
    df_Case_SCH_17_APPS.groupby(df_Case_SCH_17_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_17['month'] = days_per_month_17['APP DATE'].dt.strftime('%Y-%m')
days_per_month_17 = days_per_month_17[['month', 'clinic_days']]
print("\nSchool 17:")
print(days_per_month_17)

# ---------- SCHOOL 19 ----------
df_Case_SCH_19_APPS['APP DATE'] = pd.to_datetime(df_Case_SCH_19_APPS['APP DATE'], format='%m/%d/%y', errors='coerce')

days_per_month_19 = (
    df_Case_SCH_19_APPS.groupby(df_Case_SCH_19_APPS['APP DATE'].dt.to_period('M'))['APP DATE']
    .nunique()
    .reset_index(name='clinic_days')
)
days_per_month_19['month'] = days_per_month_19['APP DATE'].dt.strftime('%Y-%m')
days_per_month_19 = days_per_month_19[['month', 'clinic_days']]
print("\nSchool 19:")
print(days_per_month_19)


School 1:
      month  clinic_days
0   2012-01            2
1   2012-02            1
2   2012-03            2
3   2012-04            2
4   2012-05            2
5   2012-06            1
6   2012-07            2
7   2012-08            2
8   2012-09            1
9   2012-10            2
10  2012-11            2
11  2012-12            2
12  2013-01            2
13  2013-02            1
14  2013-03            2
15  2013-04            2
16  2013-05            1
17  2013-06            2
18  2013-07            2
19  2013-08            2
20  2013-09            1
21  2013-10            2
22  2013-11            2
23  2013-12            2

School 3:
      month  clinic_days
0   2012-01            2
1   2012-02            1
2   2012-03            2
3   2012-04            2
4   2012-05            1
5   2012-06            2
6   2012-07            2
7   2012-08            2
8   2012-09            1
9   2012-10            2
10  2012-11            2
11  2012-12            2
12  2013-01            1
13 

In [ ]:
# List of odd school DataFrame variable names
odd_df_names = [
    'days_per_month_1','days_per_month_3','days_per_month_5',
    'days_per_month_7','days_per_month_9','days_per_month_11',
    'days_per_month_13','days_per_month_15','days_per_month_17',
    'days_per_month_19'
]

odd_dfs = []
for name in odd_df_names:
    if name in globals():
        odd_dfs.append(globals()[name].copy())
    else:
        print(f"Warning: {name} not found — skipping")

if not odd_dfs:
    raise ValueError("No odd school DataFrames found.")

# Combine them, then aggregate
combined_odd = pd.concat(odd_dfs, ignore_index=True)

clinic_days_total_odd = (
    combined_odd.groupby('month', as_index=False)['clinic_days']
                .sum()
                .rename(columns={'clinic_days': 'total_clinic_days'})
                .sort_values('month')
)

print("\n Total Clinic Days per Month (Odd Schools):")
print(clinic_days_total_odd)



 Total Clinic Days per Month (Odd Schools):
      month  total_clinic_days
0   2012-01                 18
1   2012-02                 16
2   2012-03                 18
3   2012-04                 17
4   2012-05                 18
5   2012-06                 17
6   2012-07                 18
7   2012-08                 18
8   2012-09                 16
9   2012-10                 18
10  2012-11                 18
11  2012-12                 17
12  2013-01                 18
13  2013-02                 16
14  2013-03                 17
15  2013-04                 18
16  2013-05                 18
17  2013-06                 16
18  2013-07                 18
19  2013-08                 18
20  2013-09                 17
21  2013-10                 18
22  2013-11                 17
23  2013-12                 18


**From the above results, we know that, historically, Van 1 serves 16 appoints per day for each school, only one school on the same date, maximum 2 clinic days and minimum 1 clinic day per month per school, and 18 clinical days maximum 16 clinical days minimum for all schools per month.**

### Estimate Appointment needed in the next month

Across all schools' datasets, based on 'APP DATE' all schools have ended their records in the month of December 2013. Based on this context, I assume that for the 'next month' in the project description, we are estimating for the month of Janurary of 2014.

For this section, we will single out patient info for each school that has 'DUE BACK' dates in 1/2014 across all schools

In [ ]:
# Target month
start = pd.Timestamp(2014, 1, 1)
end   = pd.Timestamp(2014, 2, 1)  # exclusive upper bound

filtered_by_school = {}  # {school_num: DataFrame}

for i in range(20):
    varname = f"df_Case_SCH_{i}_APPS"
    if varname not in globals():
        print(f"Warning: {varname} not found — skipping")
        continue

    df_i = globals()[varname].copy()

    # Ensure DUE BACK is datetime; leave everything else as-is
    if not pd.api.types.is_datetime64_any_dtype(df_i.get('DUE BACK')):
        df_i['DUE BACK'] = pd.to_datetime(df_i['DUE BACK'], errors='coerce')

    # Keep rows with DUE BACK in Jan 2014
    mask = (df_i['DUE BACK'] >= start) & (df_i['DUE BACK'] < end)
    df_jan2014 = df_i.loc[mask].copy()

    filtered_by_school[i] = df_jan2014

    # Optional: quick count per school
    print(f"School {i}: {df_jan2014.shape[0]} rows due back in Jan 2014")

# ---- Combined DataFrame across all schools (with school_num column) ----
combined_jan2014 = []
for i, df_i in filtered_by_school.items():
    if not df_i.empty:
        _tmp = df_i.copy()
        _tmp['school_num'] = i
        combined_jan2014.append(_tmp)

if combined_jan2014:
    combined_jan2014 = pd.concat(combined_jan2014, ignore_index=True)
else:
    combined_jan2014 = pd.DataFrame()  # empty if none found

# Peek at the result
print("\nCombined Jan 2014 rows across all schools:")
print(combined_jan2014.head())


School 0: 31 rows due back in Jan 2014
School 1: 26 rows due back in Jan 2014
School 2: 29 rows due back in Jan 2014
School 3: 25 rows due back in Jan 2014
School 4: 18 rows due back in Jan 2014
School 5: 13 rows due back in Jan 2014
School 6: 29 rows due back in Jan 2014
School 7: 23 rows due back in Jan 2014
School 8: 32 rows due back in Jan 2014
School 9: 26 rows due back in Jan 2014
School 10: 27 rows due back in Jan 2014
School 11: 26 rows due back in Jan 2014
School 12: 26 rows due back in Jan 2014
School 13: 34 rows due back in Jan 2014
School 14: 21 rows due back in Jan 2014
School 15: 14 rows due back in Jan 2014
School 16: 19 rows due back in Jan 2014
School 17: 14 rows due back in Jan 2014
School 18: 21 rows due back in Jan 2014
School 19: 23 rows due back in Jan 2014

Combined Jan 2014 rows across all schools:
      VAN   APP DATE  APP TIME        PATIENT  LAST SEEN   LAST DIAG  \
0  Van #0 2013-10-02  10:00:00   Patient #888 2013-08-28    Improved   
1  Van #0 2013-10-02  

I will merge this with severity level

In [ ]:
# 1) Clean keys on both DataFrames (safe, keeps IDs as-is)
df_Case_PatientSeverity['PATIENT'] = df_Case_PatientSeverity['PATIENT'].astype(str).str.strip()
combined_jan2014['PATIENT'] = combined_jan2014['PATIENT'].astype(str).str.strip()

# 2) If df_Case_PatientSeverity has duplicates, reduce to one row per patient.
#    (Change the aggregation if you prefer 'first'.)
severity_dedup = (
    df_Case_PatientSeverity
    .sort_index()                   # keep original order; or use another sort if needed
    .drop_duplicates(subset=['PATIENT'], keep='last')  # last wins on ties
    [['PATIENT','SEVERITY']]
)

# 3) Left-merge severity into your combined table
combined_with_severity = combined_jan2014.merge(
    severity_dedup, on='PATIENT', how='left'
)

# Optional: flag who didn’t match, or fill missing with a label
missing_patients = combined_with_severity['SEVERITY'].isna().sum()
print(f"Rows without severity: {missing_patients}")

# Optional fill:
# combined_with_severity['SEVERITY'] = combined_with_severity['SEVERITY'].fillna('Unknown')

# Peek
print(combined_with_severity.head())


Rows without severity: 0
      VAN   APP DATE  APP TIME        PATIENT  LAST SEEN   LAST DIAG  \
0  Van #0 2013-10-02  10:00:00   Patient #888 2013-08-28    Improved   
1  Van #0 2013-10-02  11:30:00  Patient #1107 2013-08-12  Controlled   
2  Van #0 2013-10-02  12:00:00  Patient #1001 2013-08-12  Controlled   
3  Van #0 2013-10-02  13:00:00  Patient #1139 2013-08-28  Controlled   
4  Van #0 2013-10-02  13:30:00   Patient #644 2013-08-28  Controlled   

  LAST DUE BACK CURRENT DIAG   DUE BACK COMMENT HOSP. ER-VISIT DAYS MISSED  \
0    2013-09-28   Controlled 2014-01-02     NaN     0        0           0   
1    2013-11-12   Controlled 2014-01-02     NaN     0        0           0   
2    2013-11-12   Controlled 2014-01-02     NaN     0        0           0   
3    2013-11-28   Controlled 2014-01-02     NaN     0        0           0   
4    2013-11-28   Controlled 2014-01-02     NaN     0        0           0   

   school_num          SEVERITY  
0           0  MildIntermittent  
1    

In [ ]:
(combined_with_severity['HOSP.'] == 1).sum()

np.int64(1)

In [ ]:
(combined_with_severity['ER-VISIT'] == 1).sum()


np.int64(4)

In [ ]:
(combined_with_severity['CURRENT DIAG'] == 'Worsened').sum()

np.int64(32)

In [ ]:
# drop the columns I don't need: LAST SEEN, LAST DUE BACK, COMMENT, DAYS MISSED

combined_with_severity = combined_with_severity.drop(columns=['LAST SEEN', 'LAST DUE BACK', 'COMMENT', 'DAYS MISSED'])

Creating a dsitance matrix for the schools

In [ ]:
# Given school location data
data = {
    'School': [f"School {i}" for i in range(20)],
    'X': [13, 67, 56, 54, 23, 89, 48, 32, 97, 14, 47, 11, 95, 81, 7, 84, 5, 35, 56, 73],
    'Y': [42, 34, 94, 86, 93, 98, 90, 71, 13, 46, 22, 4, 8, 67, 74, 14, 25, 36, 59, 41]
}

df_loc = pd.DataFrame(data)

# Compute pairwise Euclidean distance matrix
coords = df_loc[['X', 'Y']].values
dist_matrix = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=-1)

# Convert to a readable DataFrame
dist_df = pd.DataFrame(dist_matrix, index=df_loc['School'], columns=df_loc['School'])

print("\n Distance Matrix Between Schools:")
print(dist_df.round(2))



 Distance Matrix Between Schools:
School     School 0  School 1  School 2  School 3  School 4  School 5  \
School                                                                  
School 0       0.00     54.59     67.48     60.14     51.97     94.40   
School 1      54.59      0.00     61.00     53.60     73.60     67.68   
School 2      67.48     61.00      0.00      8.25     33.02     33.24   
School 3      60.14     53.60      8.25      0.00     31.78     37.00   
School 4      51.97     73.60     33.02     31.78      0.00     66.19   
School 5      94.40     67.68     33.24     37.00     66.19      0.00   
School 6      59.41     59.14      8.94      7.21     25.18     41.77   
School 7      34.67     50.93     33.24     26.63     23.77     63.07   
School 8      88.87     36.62     90.79     84.72    108.98     85.38   
School 9       4.12     54.34     63.78     56.57     47.85     91.26   
School 10     39.45     23.32     72.56     64.38     74.95     86.83   
School 11     38

## Assumptions & Data Need


### Assumptions

*   We don't care about the overall costs. We care about maximizing serving appoinments scheduled, especially for those that are more severe.
*   The appointments of each school is taken care of by one van.
*   Each service is assumed to fully satisfy the patients' full medical need.
*   High-severity patients receive higher priority through weighted benefits in the objective function
“Must-serve” patients (with ER visit, hospitalization, or worsened diagnosis)

*   “Must-serve” patients are the ones with ER visit, hospitalization, or worsened diagnosis, since these conditions indicate that they are very in need of medical attention.
*   Only two vans are available --- no back up plans

*  Based on the historical data, a van can only serve one school per day, and travel between schools on the same day is not allowed.
*   Clinic capacity is limited to 16 patients per van per day based on the historical data.


*   A school can only be visited up to 2 clinic days per month by the same van, also based on the historical data.
*   Each school is assigned to exactly one van for the entire month (no van switching allowed), also based on the historical data.


*   Distance between schools is assumed to reflect travel difficulty, and assigning far-apart schools to the same van is discouraged using a quadratic penalty based on a 90-mile distance threshold.
*   Vans have upper monthly limits on clinic days (Van 0: 19 days, Van 1: 18 days) based on the historical data.


*   All demand and scheduling decisions are known in advance (perfect information), with no alterations in patient availability or school schedules.
*   Service can only occur on predefined “DUE BACK” dates of that month.

*   All model parameters (severity, diagnoses, ER visits, etc.) are assumed correct and up to date, with no data entry errors.
*   By "Next month" we mean specifically the month of January 2014, since the data provided showed the appointments dates for all schools are all in December 2013.



### Data Need


*   Costs for booking each van on each day for each school.
*   Budget constraints.
*   Whether any patients or schools have accessibility needs, and whether both vans are properly equipped and staffed to accommodate those needs.





## 3. Modeling

Total number of patients wating to be served

In [ ]:
len(combined_with_severity)

477

Ideally, I will have enough capacity to serve 477 patients in the next month across all schools.

# Algebraic Closed Form Formulation

## Sets

* V : set of vans (e.g., {0,1})
* S : set of schools
* P : set of patients
* D : set of clinic days (unique normalized “DUE BACK” dates)

## Parameters
* school(p) ∈ S : school of patient p
* w(p) ∈ {1,2,3,4} : severity weight for patient p
  mapping: MildIntermittent→1, MildPersistent→2, ModeratePersistent→3, SeverePersistent→4
* cap = 16 : max patients per van-day
* maxDaysSchool = 2 : max clinic days a given van can serve a given school in the month
* maxDaysVan(0) = 19, maxDaysVan(1) = 18 : monthly day limits per van
* dist(s1,s2) ≥ 0 : distance between schools s1 and s2
* F = { (s1,s2) ∈ S×S : s1 < s2 and dist(s1,s2) > 90 }  (pairs considered “far apart” if the distance between them are greater than 90. This is a arbitrary theshold I came up with by eyeing the distance matrix between the schools)
* α > 0 : penalty weight for assigning “far apart” schools to the same van for the month (your code uses α = 5)
* M ⊆ P : must-serve patients
  M = { p ∈ P : (ER-VISIT(p)=1) OR (HOSP.(p)=1) OR (CURRENT DIAG(p)=“Worsened”) }
* S_demand ⊆ S : schools with at least one patient in P

## Decision variables

* x[p,v,d] ∈ {0,1} : 1 if patient p is scheduled with van v on day d; 0 otherwise
* y[s,v,d] ∈ {0,1} : 1 if van v serves school s on day d; 0 otherwise
* z[s,v]   ∈ {0,1} : 1 if school s is assigned to van v for the whole month; 0 otherwise

## Objective

Maximize severity coverage based on severity weights while discouraging assigning far-apart schools (with distance over 90)to the same van:

Maximize
Σ_{p∈P} Σ_{v∈V} Σ_{d∈D} w(p) * x[p,v,d]
− α * Σ_{v∈V} Σ_{(s1,s2)∈F} z[s1,v] * z[s2,v]

## Constraints

(1) Serve a patient only if the van is at that patient’s school that day
For all p ∈ P, v ∈ V, d ∈ D, let s = school(p):
x[p,v,d] ≤ y[s,v,d]

(2) Patient scheduled at most once in the month:
For all p ∈ P:
Σ_{v∈V} Σ_{d∈D} x[p,v,d] ≤ 1

(3) Must-serve patients (ER-VISIT==1 OR HOSP.==1 OR CURRENT DIAG==“Worsened”) are scheduled exactly once:
For all p ∈ M:
Σ_{v∈V} Σ_{d∈D} x[p,v,d] = 1

(4) Van-day capacity:
For all v ∈ V, d ∈ D:
Σ_{p∈P} x[p,v,d] ≤ cap

(5) A van serves at most one school per day:
For all v ∈ V, d ∈ D:
Σ_{s∈S} y[s,v,d] ≤ 1

(6) At most two clinic days at a school by a given van:
For all v ∈ V, s ∈ S:
Σ_{d∈D} y[s,v,d] ≤ maxDaysSchool

(7) Monthly day limits per van:
Σ_{s∈S} Σ_{d∈D} y[s,0,d] ≤ 19
Σ_{s∈S} Σ_{d∈D} y[s,1,d] ≤ 18

(8) Day service only if the van is assigned to that school for the month:
For all s ∈ S, v ∈ V, d ∈ D:
y[s,v,d] ≤ z[s,v]

(9) One-van-per-school for the month:
For all s ∈ S_demand:
Σ_{v∈V} z[s,v] = 1
For all s ∈ S \ S_demand:
Σ_{v∈V} z[s,v] ≤ 1

(10) Variable domains:
x[p,v,d] ∈ {0,1}, y[s,v,d] ∈ {0,1}, z[s,v] ∈ {0,1}

I wish but cannot perform linear relaxation on the model because my decision variables are binary, and it does not really make sense to make them continuous between 0 and 1

In [ ]:
# Data:
df = combined_with_severity.copy()

severity_map = {
    'MildIntermittent': 1,
    'MildPersistent': 2,
    'ModeratePersistent': 3,
    'SeverePersistent': 4
}

df['severity_weight'] = (
    df['SEVERITY'].str.strip()
    .map(severity_map)
    .fillna(1)  # fallback if any unexpected category appears
)

VANS = [0, 1]
SCHOOLS = sorted(df['school_num'].unique())
PATIENTS = df.index.tolist()
DAYS = sorted(df['DUE BACK'].dt.normalize().unique())

school_of_patient = df['school_num'].to_dict()
severity = df['severity_weight'].to_dict()

m = gp.Model("Van_Scheduling", env=env)

# Decision variables
x = m.addVars(PATIENTS, VANS, DAYS, vtype=GRB.BINARY, name="x")  # assign patient
y = m.addVars(SCHOOLS, VANS, DAYS, vtype=GRB.BINARY, name="y")  # assign school per van-day

# ---- NEW: One van per school (no switching vans between dates) ----

# Schools that actually have patients in Jan 2014
schools_with_patients = sorted(df['school_num'].unique())

# School-van assignment decision (for whole month)
z = m.addVars(SCHOOLS, VANS, vtype=GRB.BINARY, name="z")

# Day service only allowed if van is assigned to that school
for s in SCHOOLS:
    for v in VANS:
        for d in DAYS:
            m.addConstr(y[s,v,d] <= z[s,v])

# Each school can only be served by ONE van for the month
for s in SCHOOLS:
    if s in schools_with_patients:
        # must choose exactly one van if school has demand
        m.addConstr(gp.quicksum(z[s,v] for v in VANS) == 1)
    else:
        # optional if school has no Jan patients — allow 0 or 1
        m.addConstr(gp.quicksum(z[s,v] for v in VANS) <= 1)

# 1) Build the main linear objective explicitly
linear_obj = gp.quicksum(severity[p] * x[p,v,d] for p in PATIENTS for v in VANS for d in DAYS)

# 2) Build ONE quadratic penalty expression for far-apart schools assigned to the same van
PENALTY_WEIGHT = -5  # tiny penalty since we maximize

penalty_pairs = []
for v in VANS:
    for s1 in SCHOOLS:
        for s2 in SCHOOLS:
            if s1 < s2:  # avoid double-counting
                if dist_df.loc[f"School {s1}", f"School {s2}"] > 90:
                    penalty_pairs.append(PENALTY_WEIGHT * z[s1, v] * z[s2, v])

quad_penalty = gp.quicksum(penalty_pairs) if penalty_pairs else 0.0

# Objective: Max severity coverage + PENALIZING schools that are too far away (I set the threshold to more than 90)
m.setObjective(linear_obj + quad_penalty, GRB.MAXIMIZE)

# Constraints
# School consistency
for p in PATIENTS:
    s = school_of_patient[p]
    for v in VANS:
        for d in DAYS:
            m.addConstr(x[p,v,d] <= y[s,v,d])

# Must-serve patients: ER visit OR hospitalization OR worsened diagnosis
must_serve_mask = (
    (df['ER-VISIT'].fillna(0) == 1) |
    (df['HOSP.'].fillna(0) == 1) |
    (df['CURRENT DIAG'].fillna('').str.strip().str.casefold() == 'Worsened')
)
MUST_SERVE = df.index[must_serve_mask].tolist()

# Patient only once
for p in PATIENTS:
    m.addConstr(gp.quicksum(x[p,v,d] for v in VANS for d in DAYS) <= 1)

# Capacity 16 per van-day
for v in VANS:
    for d in DAYS:
        m.addConstr(gp.quicksum(x[p,v,d] for p in PATIENTS) <= 16)

# Van only one school per day
for v in VANS:
    for d in DAYS:
        m.addConstr(gp.quicksum(y[s,v,d] for s in SCHOOLS) <= 1)

# Max 2 clinic days per school per van
for v in VANS:
    for s in SCHOOLS:
        m.addConstr(gp.quicksum(y[s,v,d] for d in DAYS) <= 2)

# Max clinic days per van per month
m.addConstr(gp.quicksum(y[s,0,d] for s in SCHOOLS for d in DAYS) <= 19)  # van 0
m.addConstr(gp.quicksum(y[s,1,d] for s in SCHOOLS for d in DAYS) <= 18)  # van 1

# Required for non-convex quadratic
m.Params.NonConvex = 2

m.Params.TimeLimit = 100       # stop after 100 seconds

m.optimize()

# Extract assignment solution
assignments = []
for p in PATIENTS:
    for v in VANS:
        for d in DAYS:
            if x[p,v,d].X > 0.5:
                assignments.append({
                    "van": v,
                    "school": school_of_patient[p],
                    "patient": df.loc[p, 'PATIENT'],
                    "date": d,
                    "severity": df.loc[p, 'SEVERITY']
                })

schedule_solution = pd.DataFrame(assignments)
display(schedule_solution)


Set parameter NonConvex to value 2
Set parameter TimeLimit to value 100
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  100
NonConvex  2

Academic license 2709572 - for non-commercial use only - registered to mg___@columbia.edu
Optimize a model with 31477 rows, 30854 columns and 124536 nonzeros (Max)
Model fingerprint: 0x8c3cd0fa
Model has 29574 linear objective coefficients
Model has 42 quadratic objective terms
Variable types: 0 continuous, 30854 integer (30854 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -70.0000000
Presolve removed 20 rows and 20 columns
Presolve time: 0.65s

,van,school,patient,date,severity
0,0,0,Patient #888,2014-01-05,MildIntermittent
1,0,0,Patient #1107,2014-01-10,MildPersistent
2,0,0,Patient #1001,2014-01-05,MildIntermittent
3,0,0,Patient #1139,2014-01-10,MildIntermittent
4,0,0,Patient #644,2014-01-10,MildIntermittent
...,...,...,...,...,...
470,0,19,Patient #1352,2014-01-09,ModeratePersistent
471,0,19,Patient #1350,2014-01-17,ModeratePersistent
472,0,19,Patient #272,2014-01-17,SeverePersistent
473,0,19,Patient #22,2014-01-17,ModeratePersistent


# Result

After optimization, Van 1 is assigned to Schools 0, 7, 8, 9, 10, 11, 12, 15, 17, and 18 — exactly 10 out of 20 schools, which is a very balanced result — while Van 0 serves the remaining schools. Only two patients were not scheduled. Both have well-controlled, Mild Intermittent asthma, which is the least severe category; therefore, the current schedule sufficiently meets medical priorities and operational constraints.

The full scheduling results have been exported to an Excel file.

In [ ]:
# Which school is served by which van in the optimized solution?
school_van_map = (
    schedule_solution.groupby('school')['van']
                     .agg(lambda x: x.mode()[0])  # van that appears for that school
                     .reset_index()
                     .sort_values('school')
)

school_van_map.columns = ['school_num', 'assigned_van']
print("\n=== School → Van Assignment ===")
print(school_van_map)



=== School → Van Assignment ===
    school_num  assigned_van
0            0             0
1            1             0
2            2             1
3            3             1
4            4             1
5            5             1
6            6             1
7            7             1
8            8             0
9            9             0
10          10             0
11          11             0
12          12             0
13          13             1
14          14             1
15          15             0
16          16             1
17          17             1
18          18             0
19          19             0


In [ ]:
# Clean and sort final schedule
schedule_sorted = schedule_solution.copy()
schedule_sorted['date'] = pd.to_datetime(schedule_sorted['date'])
schedule_sorted = schedule_sorted.sort_values(["van", "date", "patient"]).reset_index(drop=True)

print("\n FINAL JAN 2014 SCHEDULE (Sorted by Van & Date)")
print(schedule_sorted)



 FINAL JAN 2014 SCHEDULE (Sorted by Van & Date)
     van  school        patient       date            severity
0      0       1  Patient #1056 2014-01-01  ModeratePersistent
1      0       1  Patient #1085 2014-01-01  ModeratePersistent
2      0       1   Patient #124 2014-01-01  ModeratePersistent
3      0       1  Patient #1303 2014-01-01      MildPersistent
4      0       1  Patient #1456 2014-01-01    MildIntermittent
..   ...     ...            ...        ...                 ...
470    1       6   Patient #885 2014-01-29      MildPersistent
471    1       6   Patient #885 2014-01-29      MildPersistent
472    1       6   Patient #964 2014-01-29  ModeratePersistent
473    1       4   Patient #553 2014-01-30      MildPersistent
474    1       4   Patient #831 2014-01-30      MildPersistent

[475 rows x 5 columns]


Convert this to an excel format:

In [ ]:
file_name = "Van_Schedule_Solution.xlsx"
schedule_sorted.to_excel(file_name, index=False)

print(f" Excel file saved as: {file_name}")

 Excel file saved as: Van_Schedule_Solution.xlsx


In [ ]:
# Unsatisfied Demand per School
scheduled_patients = set(schedule_solution['patient'])
jan_patients = combined_with_severity.copy()
jan_patients['scheduled'] = jan_patients['PATIENT'].isin(scheduled_patients)

unscheduled = jan_patients[jan_patients['scheduled'] == False]

unsatisfied_by_school = (
    unscheduled.groupby("school_num")['PATIENT']
    .count()
    .reset_index(name="unscheduled_patients")
)

print("\n Unsatisfied Demand per School")
print(unsatisfied_by_school)




 Unsatisfied Demand per School
   school_num  unscheduled_patients
0          13                     2


In [ ]:
print("\nUnscheduled Patients (Full List)")
print(unscheduled[['PATIENT', 'school_num']])


Unscheduled Patients (Full List)
           PATIENT  school_num
333  Patient #1113          13
345  Patient #1354          13


Check if these Patients have any of these conditions: had an ER visit, hospitalized, got worsened or were severely persistant

In [ ]:
df[df['PATIENT'].isin(['Patient #1113', 'Patient #468'])]

,VAN,APP DATE,APP TIME,PATIENT,LAST DIAG,CURRENT DIAG,DUE BACK,HOSP.,ER-VISIT,school_num,SEVERITY,severity_weight
333,Van #1,2013-10-14,10:00:00,Patient #1113,Controlled,Controlled,2014-01-14,0,0,13,MildIntermittent,1
352,Van #1,2013-10-31,14:30:00,Patient #468,Controlled,Controlled,2014-01-31,0,0,13,MildIntermittent,1


# Objective Selection

My objective function prioritizes serving higher-severity patients by assigning larger weights to their coverage. At the same time, it includes a soft penalty for assigning schools that are very far apart to the same van using a 90-mile threshold. The primary strength of this design is that it balances clinical importance (serving the most severe patients first) with logistical feasibility (avoiding inefficient van assignments). It is both rational and relatively straightforward to implement. However, there are limitations. The use of a hard 90-mile cutoff ignores variations in distance below that threshold, meaning some meaningful logistic differences are not captured. Additionally, due to limited data, cost considerations are not included, even though operational costs would be a major factor in real-world planning.
  

An alternative objective is a cost minimization approach. This can be valuable when budget constraints are tight or when schools and patients face financial challenges, making the solution more realistic for resource-constrained operations. The drawback, however, is that cost-focused models may fail to prioritize patients with more severe medical needs, potentially leaving high-risk individuals unserved.

